In [ ]:
import numpy as np
import os
import glob
import pandas as pd
import shutil
from tqdm import tqdm


In [ ]:

file_path = '/data/rsg/chemistry/jyim/projects/flow-matching/notebooks/filenames.txt'
all_genie_files = []
with open(file_path, 'r') as file:
    for line in file:
        all_genie_files.append(line.strip())
all_genie_files = set(all_genie_files)

In [ ]:
scope_dir = '/data/rsg/chemistry/jyim/projects/genie/data/pdbstyle-2.08'
write_dir = '/data/rsg/chemistry/jyim/large_data/scope'
all_file_paths = glob.glob(
    os.path.join(scope_dir, '**/*.ent')
)

for file_path in tqdm(all_file_paths):
    filename = os.path.basename(file_path)
    if filename.replace('.ent', '') in all_genie_files:
        write_path = os.path.join(write_dir, filename.replace('.ent', '.pdb'))
        shutil.copy(file_path, write_path)
